In [8]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np 
import matplotlib.animation as animation
import cartopy.feature as cfeature

In [9]:
file = '/glade/work/mollyw/Projects/sea_ice_covariability/data/cryosat-2/cryosat-2/cs2_landy_data.nc'

In [10]:
ds = xr.open_dataset(file)

In [11]:
ds

<xarray.Dataset>
Dimensions:    (time: 236, x: 100, y: 100)
Coordinates:
    latitude   (x, y) float64 ...
    longitude  (x, y) float64 ...
  * time       (time) datetime64[ns] 2010-10-09 2010-10-24 ... 2020-07-24
Dimensions without coordinates: x, y
Data variables:
    algorithm  (time) float64 ...
    sithick    (time, x, y) float64 ...
    sit_unc    (time, x, y) float64 ...
    siconc     (time, x, y) float64 ...
    ice_type   (time, x, y) float64 ...
    grid_area  (x, y) float64 ...
    siarea     (time, x, y) float64 ...
    sivol      (time, x, y) float64 ...
    SIA        (time) float64 ...
    SIV        (time) float64 ...
Attributes: (12/36)
    Dataset:                   Sea ice thickness observations derived from Cr...
    Date_Created:              18-Aug-2021
    Source_Altimetry:          Cryosat-2 SIRAL Baseline-D L1B SAR and SARIn W...
    Source_Snow:               Distributed Snow Evolution Model (SnowModel-LG...
    Source_Ice_Concentration:  OSI-SAF Global Sea Ice Concentration (OSI-401-b)
    Source_Ice_Type:           NSIDC Sea Ice Age v4.1
    ...                        ...
    publisher_name:            NERC EDS UK Polar Data Centre
    publisher_type:            institution
    publisher_email:           pdc@bas.ac.uk
    publisher_url:             https://www.bas.ac.uk/data/uk-pdc/
    references:                Please cite the following publications if usin...
    citation:                  Landy, J., & Dawson, G. (2022). Year-round Arc...

In [12]:
def convert_coords(lat, lon, og_data, ccrs_grid):
    """ convert data to appropriate coordinates for ccrs plot """
    
    grid = ccrs_grid
    coords = grid.transform_points(ccrs.PlateCarree(), 
                                   np.array(lon), 
                                   np.array(lat))
    
    xs = np.ma.masked_invalid(coords[..., 0])
    ys = np.ma.masked_invalid(coords[..., 1])
    data = np.ma.masked_invalid(og_data)
    data.mask = np.logical_or(data.mask, xs.mask, ys.mask)
    
    xs = xs.filled(0)
    ys = ys.filled(0)
    
    return xs,ys,data

In [13]:
data = ds.sithick[0]
lat = ds.sithick[0].latitude
lon = ds.sithick[0].longitude
nps = ccrs.NorthPolarStereo()
    
xs,ys,plot_data=convert_coords(lat, lon, data, nps)

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols= 1, subplot_kw={'projection': ccrs.NorthPolarStereo()})
axes.add_feature(cfeature.LAND,facecolor = 'gainsboro', zorder=6)
im = axes.contourf(xs, ys, data, vmin = 0, vmax = 2, transform=ccrs.NorthPolarStereo(), cmap = 'Blues_r')
plt.colorbar(im, ax = axes)
axes.set_title(ds.time[0].values)

# fig = plt.figure()
# i=0
# im = plt.imshow(arr[0], animated=True)
def animate(i):
    im.collections = []
    xs,ys,d = convert_coords(ds.latitude, ds.longitude, ds.sithick[i], ccrs.NorthPolarStereo())
    axes.contourf(xs, ys, d, vmin = 0, vmax = 2, transform=ccrs.NorthPolarStereo(), cmap = 'Blues_r')
    plt.colorbar(im, ax = axes)
    axes.set_title(ds.time[i].values)

ani = animation.FuncAnimation(fig, animate,len(ds.time))



writergif = animation.PillowWriter(fps=5)
ani.save('filename.gif',writer=writergif)